In [4]:
# !pip install opendatasets
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hefryanesti","key":"f3cca610517ba5995bdb9f4bc3ce33d1"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d hefryanesti/gfrac-resources-dataset

100% 6.88G/6.90G [01:13<00:00, 127MB/s]
100% 6.90G/6.90G [01:13<00:00, 101MB/s]


In [7]:
!unzip -q /content/gfrac-resources-dataset.zip

In [8]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import xarray
import numpy as np
import pandas as pd

In [1]:
import xarray
import numpy as np
import pandas as pd
#Read Data
#States
max_year = 5
old_settings = np.seterr(all='ignore')
path_LUH2 = '/content/states.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="h5netcdf", decode_times=False)
luh2_states_worldwide = luh2_states.isel(time=slice(1120, 1173))
luh2_states_worldwide['time'] = pd.date_range(start="1970-01-01", end="2022-01-01", freq='YS')

#Transition
transition_LUH2 = "/content/drive/MyDrive/RA - Hefri/Output Dataset/NATVEG BIOMASS & LUH2/LUH2_sum_natveg_FROM_1970_2022_run3.nc"
luh2_trans_worldwide = xarray.open_dataset(transition_LUH2, engine="h5netcdf")

#add static data for carea
static = '/content/staticData_quarterdeg.nc'
luh2_static = xarray.open_dataset(static, engine="h5netcdf")

#ccode baru
staticcode = "/content/gpw_v4_national_identifier_grid_rev11_15_min_finall.nc"
luh2_static2 = xarray.open_dataset(staticcode, engine="h5netcdf")

#join the code
luh2_static = xarray.merge([luh2_static2, luh2_static], join='right', compat="override")

#convert **ccode** to Country Name
country_code = pd.read_excel("/content/ISO-3166-Country-Code.xlsx", engine="openpyxl")

ccode_iso = list(country_code['country-code'])
csub_iso = list(country_code['sub-region'])
cregion_iso = list(country_code['region'])
cname_iso = list(country_code['name'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh2_static['ccode'].to_numpy().astype('int64')

ccode_convert = np.zeros((720, 1440), dtype="<U64")
# ccode_convert_cregion = np.zeros((720, 1440), dtype="<U64")
# ccode_convert_sub = np.zeros((720, 1440), dtype="<U64")

for i in range(720):
    for j in range(1440):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
            # ccode_convert_cregion[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
            # ccode_convert_sub[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"
            # ccode_convert_cregion[i][j] = "Ocean"
            # ccode_convert_sub[i][j] = "Ocean"

country = xarray.Dataset({"country": (["lat", "lon"], ccode_convert)},
                          coords={ "lon":luh2_states_worldwide.coords["lon"].to_numpy(), "lat":luh2_states_worldwide.coords["lat"].to_numpy()})
# subregion = xarray.Dataset({"sub-region": (["lat", "lon"], ccode_convert_sub)},
#                           coords={ "lon":luh2_states_worldwide.coords["lon"].to_numpy(), "lat":luh2_states_worldwide.coords["lat"].to_numpy()})
# region = xarray.Dataset({"region": (["lat", "lon"], ccode_convert_cregion)},
#                           coords={ "lon":luh2_states_worldwide.coords["lon"].to_numpy(), "lat":luh2_states_worldwide.coords["lat"].to_numpy()})

In [2]:

#natveg
natveg_all_carbon_density = xarray.open_dataset("/content/drive/MyDrive/RA - Hefri/Output Dataset/NATVEG BIOMASS & LUH2/natveg_ALL-1970-2100_run1.nc", engine="h5netcdf")

#agri
pathc3ann = "/content/drive/MyDrive/Interpolation/CARBON-STOCK-CROP-C3ANN-INTERPOLATION-WORLDWIDE-1970_2100.nc"
c3ann_carbon_density = xarray.open_dataset(pathc3ann, engine="h5netcdf")
pathc4ann = "/content/drive/MyDrive/Interpolation/CARBON-STOCK-CROP-C4ANN-INTERPOLATION-WORLDWIDE-1970_2100.nc"
c4ann_carbon_density = xarray.open_dataset(pathc4ann, engine="h5netcdf")
pathc3per = "/content/drive/MyDrive/Interpolation/CARBON-STOCK-CROP-C3PER-INTERPOLATION-WORLDWIDE-1970_2100.nc"
c3per_carbon_density = xarray.open_dataset(pathc3per, engine="h5netcdf")
pathc4per = "/content/drive/MyDrive/Interpolation/CARBON-STOCK-CROP-C4PER-INTERPOLATION-WORLDWIDE-1970_2100.nc"
c4per_carbon_density = xarray.open_dataset(pathc4per, engine="h5netcdf")
pathc3nfx = "/content/drive/MyDrive/Interpolation/CARBON-STOCK-CROP-C3NFX-INTERPOLATION-WORLDWIDE-1970_2100.nc"
c3nfx_carbon_density = xarray.open_dataset(pathc3nfx, engine="h5netcdf")
pathpastr = "/content/drive/MyDrive/Interpolation/CARBON-STOCK-PASTURE-INTERPOLATION-WORLDWIDE-1970_2100.nc"
pastr_carbon_density = xarray.open_dataset(pathpastr, engine="h5netcdf")
#range
pathrange = "/content/drive/MyDrive/Interpolation/CARBON-STOCK-RANGE-INTERPOLATION-WORLDWIDE-1970_2100.nc"
range_carbon_density = xarray.open_dataset(pathrange, engine="h5netcdf")


In [3]:

# Carbon Density
#Biomass
carbon_density_natveg_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3ann_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c4ann_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3per_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c4per_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3nfx_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_pastr_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_range_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_all_bio = np.zeros((max_year, 720, 1440), dtype="float32")

#Soil
carbon_density_natveg_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3ann_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c4ann_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3per_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c4per_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3nfx_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_pastr_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_range_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_all_soil = np.zeros((max_year, 720, 1440), dtype="float32")

# Carbon emission in every years
carbon_emit_harv_biomass = np.zeros((max_year, 5, 720, 1440), dtype="float32")
carbon_emit_harv_soil = np.zeros((max_year, 5, 720, 1440), dtype="float32")
#1 LUC Crops (dimensi 20 karena 4 sumber: primf, secdf, primn, secdn --> menuju 5 tujuan: c3ann, c4ann, c3per, c4per, c3nfx)
'''
temp_carbon_emit_trans_LUC_crops_bio = np.zeros((max_year, 20, 720, 1440), dtype="float64")
temp_carbon_emit_trans_LUC_crops_soil = np.zeros((max_year, 20, 720, 1440), dtype="float64")
'''
#versi setelah elke meeting 1 sumber: NATVEG to 5 tujuan: c3ann, c4ann, c3per, c4per, c3nfx
temp_carbon_emit_trans_LUC_crops_bio = np.zeros((max_year, 5, 720, 1440), dtype="float64")
temp_carbon_emit_trans_LUC_crops_soil = np.zeros((max_year, 5, 720, 1440), dtype="float64")

#2 carbon emission LUC to pasture
#2a1 carbon emission forest to pasture
temp_carbon_emit_trans_LUC_pasture_bio = np.zeros((max_year, 720, 1440), dtype="float32")
temp_carbon_emit_trans_LUC_pasture_soil = np.zeros((max_year, 720, 1440), dtype="float32")
#2a2 carbon emission forest to rangeland
temp_carbon_emit_trans_LUC_range_bio = np.zeros((max_year, 720, 1440), dtype="float32")
temp_carbon_emit_trans_LUC_range_soil = np.zeros((max_year, 720, 1440), dtype="float32")

# Akumulatif carbon emission
#BIOMASS
# temp_harvest_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_LUC_crops_increment_biomass = np.zeros((max_year, 5, 720, 1440), dtype="float32")
temp_LUC_pasture_increment_biomass = np.zeros((max_year, 1, 720, 1440), dtype="float32")
temp_LUC_range_increment_biomass = np.zeros((max_year, 1, 720, 1440), dtype="float32")
# temp_abandon_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")

#SOIL
# temp_harvest_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_LUC_crops_increment_soil = np.zeros((max_year, 5, 720, 1440), dtype="float32")
temp_LUC_pasture_increment_soil = np.zeros((max_year, 1, 720, 1440), dtype="float32")
temp_LUC_range_increment_soil = np.zeros((max_year, 1, 720, 1440), dtype="float32")
# temp_abandon_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")

#Agregarsi untuk NETCDF
all_to_c3ann_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4ann_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3per_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4per_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3nfx_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_crops_bio = np.zeros((max_year, 720, 1440), dtype="float32")

# all_to_pastr_bio = np.zeros((max_year, 720, 1440), dtype="float32")
# all_to_range_bio = np.zeros((max_year, 720, 1440), dtype="float32")

all_to_c3ann_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4ann_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3per_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4per_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3nfx_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_crops_soil = np.zeros((max_year, 720, 1440), dtype="float32")

# all_to_pastr_soil = np.zeros((max_year, 720, 1440), dtype="float32")
# all_to_range_soil = np.zeros((max_year, 720, 1440), dtype="float32")

all_to_c3ann = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4ann = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3per = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4per = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3nfx = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_crops = np.zeros((max_year, 720, 1440), dtype="float32")

# all_to_pastr = np.zeros((max_year, 720, 1440), dtype="float32")
# all_to_range = np.zeros((max_year, 720, 1440), dtype="float32")

#General List
landuse_class_all = ['primf', 'primn', 'secdf', 'secdn', 'c3ann', 'c4ann', 'c3per', 'c4per', 'c3nfx', 'pastr', 'range']
landuse_class_prim_secd = ['primf', 'primn', 'secdf', 'secdn']
landuse_class_pastr_crop = ['c3ann', 'c4ann', 'c3per', 'c4per', 'c3nfx', 'pastr', 'range']
landuse_class_secd = ['secdf','secdn']
carstock_biomass = ['branches', 'leaves', 'stems', 'roots']
carstock_soil = ['litter', 'humus', 'charcoal']
land_clearing_to_crop = ['primf_to_c3ann','primf_to_c4ann','primf_to_c3per','primf_to_c4per','primf_to_c3nfx',
                        'primn_to_c3ann','primn_to_c4ann','primn_to_c3per','primn_to_c4per','primn_to_c3nfx',
                        'secdf_to_c3ann','secdf_to_c4ann','secdf_to_c3per','secdf_to_c4per','secdf_to_c3nfx',
                        'secdn_to_c3ann','secdn_to_c4ann','secdn_to_c3per','secdn_to_c4per','secdn_to_c3nfx']
land_clearing_to_pasture = ['primf_to_pastr', 'primf_to_range',
                            'primn_to_pastr', 'primn_to_range',
                            'secdf_to_pastr', 'secdf_to_range',
                            'secdn_to_pastr', 'secdn_to_range']
ajl_clearing = ['primf', 'primn', 'secdf', 'secdn']

#List for FROM analysis
#Harvest
primf_harvest = ["primf_harv"]
primn_harvest = ["primn_harv"]
secmf_harvest = ["secmf_harv"]
secyf_harvest = ["secyf_harv"]
secdn_harvest = ["secnf_harv"]

# Other Transitions
# Pasture to Crop
pastr_to_crop = ['pastr_to_c3ann','pastr_to_c4ann','pastr_to_c3per','pastr_to_c4ann','pastr_to_c3nfx']
range_to_crop = ['range_to_c3ann','range_to_c4ann','range_to_c3per','range_to_c4ann','range_to_c3nfx']
# Crop to Crop
c3ann_to_crop = ['c3ann_to_c4ann','c3ann_to_c3per','c3ann_to_c4per','c3ann_to_c3nfx']
c4ann_to_crop = ['c4ann_to_c3ann','c4ann_to_c3per','c4ann_to_c4per','c4ann_to_c3nfx']
c3per_to_crop = ['c3per_to_c3ann','c3per_to_c4ann','c3per_to_c4per','c3per_to_c3nfx']
c4per_to_crop = ['c4per_to_c3ann','c4per_to_c4ann','c4per_to_c3per','c4per_to_c3nfx']
c3nfx_to_crop = ['c3nfx_to_c3ann','c3nfx_to_c4ann','c3nfx_to_c3per','c3nfx_to_c4per']

# Crop to Pasture
c3ann_to_pastr = ['c3ann_to_pastr','c3ann_to_range']
c4ann_to_pastr = ['c4ann_to_pastr','c4ann_to_range']
c3per_to_pastr = ['c3per_to_pastr','c3per_to_range']
c4per_to_pastr = ['c4per_to_pastr','c4per_to_range']
c3nfx_to_pastr = ['c3nfx_to_pastr','c3nfx_to_range']
# Pasture to Pasture
pastr_to_pasture = ['pastr_to_range']
range_to_pasture = ['range_to_pastr']
# Land Abandonment
c3ann_to_abandon = ['c3ann_to_secdf']
c4ann_to_abandon = ['c4ann_to_secdf']
c3per_to_abandon = ['c3per_to_secdf']
c4per_to_abandon = ['c4per_to_secdf']
c3nfx_to_abandon = ['c3nfx_to_secdf']
pastr_to_abandon = ['pastr_to_secdf']
range_to_abandon = ['range_to_secdf']


In [4]:
####CARBON EMISSION CALCULATION
###Biomass
#Assign the carbon density of biomass value to each year
for time_idx in range(max_year):
    for biomass_class in carstock_biomass:
        carbon_density_natveg_bio[time_idx] += natveg_all_carbon_density['natveg'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c3ann_bio[time_idx] += c3ann_carbon_density['CROPc3ann_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c4ann_bio[time_idx] += c4ann_carbon_density['CROPc4ann_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c3per_bio[time_idx] += c3per_carbon_density['CROPc3per_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c4per_bio[time_idx] += c4per_carbon_density['CROPc4per_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c3nfx_bio[time_idx] += c3nfx_carbon_density['CROPc3nfx_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_pastr_bio[time_idx] += pastr_carbon_density['pastr_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_range_bio[time_idx] += range_carbon_density['range_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()

In [63]:
carbon_density_natveg_bio_netcdf = xarray.Dataset({
    "carbon_density_natveg_bio":(["time", "lat", "lon",], carbon_density_natveg_bio)
},coords={
        "time": pd.date_range(start='1970-01-01', end='1974-01-01', freq='YS'),
        "lat": luh2_states_worldwide.coords["lat"].to_numpy(),
        "lon": luh2_states_worldwide.coords["lon"].to_numpy(),
        "country": country['country']
    })

carbon_density_natveg_bio_netcdf

<xarray.Dataset>
Dimensions:                    (time: 5, lat: 720, lon: 1440)
Coordinates:
  * time                       (time) datetime64[ns] 1970-01-01 ... 1974-01-01
  * lat                        (lat) float64 89.88 89.62 89.38 ... -89.62 -89.88
  * lon                        (lon) float64 -179.9 -179.6 ... 179.6 179.9
    country                    (lat, lon) <U64 'ocean' 'ocean' ... 'ocean'
Data variables:
    carbon_density_natveg_bio  (time, lat, lon) float32 677.7 0.0 ... 0.0 0.0

In [64]:
df = carbon_density_natveg_bio_netcdf.to_dataframe()
df = df.reset_index()
data = pd.to_datetime(df.time.values, format='%d/%m/%Y')
df.time = data.year

In [65]:
# rata rata tiap negara
average_density_natveg_bio = np.zeros((max_year, 233, 720, 1440), dtype='float32')
# average_carbon_density_crop_c3ann_bio = np.zeros((max_year, 720, 1440), dtype='float32')
# average_carbon_density_crop_c4ann_bio = np.zeros((max_year, 720, 1440), dtype='float32')
# average_carbon_density_crop_c3per_bio = np.zeros((max_year, 720, 1440), dtype='float32')
# average_carbon_density_crop_c4per_bio = np.zeros((max_year, 720, 1440), dtype='float32')
# average_carbon_density_crop_c3nfx_bio = np.zeros((max_year, 720, 1440), dtype='float32')
# average_carbon_density_pastr_bio = np.zeros((max_year, 720, 1440), dtype='float32')
# average_carbon_density_range_bio = np.zeros((max_year, 720, 1440), dtype='float32')


In [ ]:
for time_idx, time in enumerate(range(1970, 1975)): # tahun
    for idx, country in enumerate(list(np.unique(ccode_convert))):
        average_density_natveg_bio[time_idx][idx] = df[(df['country'] == country) & (df['time'] == time)]['carbon_density_natveg_bio'].sum()/\
        len(df[(df['country'] == country) & (df['carbon_density_natveg_bio'] != 0) & (df['time'] == time)])
        # average_carbon_density_crop_c3ann_bio[time_idx] = np.where(ccode_convert == country, np.mean(carbon_density_crop_c3ann_bio[time_idx]), 0)
        # average_carbon_density_crop_c4ann_bio[time_idx] = np.where(ccode_convert == country, np.mean(carbon_density_crop_c4ann_bio[time_idx]), 0)
        # average_carbon_density_crop_c3per_bio[time_idx] = np.where(ccode_convert == country, np.mean(carbon_density_crop_c3per_bio[time_idx]), 0)
        # average_carbon_density_crop_c4per_bio[time_idx] = np.where(ccode_convert == country, np.mean(carbon_density_crop_c4per_bio[time_idx]), 0)
        # average_carbon_density_crop_c3nfx_bio[time_idx] = np.where(ccode_convert == country, np.mean(carbon_density_crop_c3nfx_bio[time_idx]), 0)
        # average_carbon_density_pastr_bio[time_idx] = np.where(ccode_convert == country, np.mean(carbon_density_pastr_bio[time_idx]), 0)
        # average_carbon_density_range_bio[time_idx] = np.where(ccode_convert == country, np.mean(carbon_density_range_bio[time_idx]), 0)

In [ ]:

# natveg_to_crops = ['sum_LUC_to_c3ann','sum_LUC_to_c4ann','sum_LUC_to_c3per','sum_LUC_to_c4per','sum_LUC_to_c3nfx']

for time_idx in range(max_year):
    for x in range(720):
        for y in range (1440):
            if not np.isnan(luh2_trans_worldwide['sum_LUC_to_c3ann'].isel(time=time_idx, lat=x, lon=y)):
                carbon_density_natveg_bio[time_idx][x][y] = average_density_natveg_bio[time_idx][x][y]


In [ ]:
natveg_to_crops = ['sum_LUC_to_c3ann','sum_LUC_to_c4ann','sum_LUC_to_c3per','sum_LUC_to_c4per','sum_LUC_to_c3nfx']

for time_idx in range(max_year):
    # for crops in natveg_to_crops:
    carbon_density_natveg_bio[time_idx] = np.where(not np.isnan(luh2_trans_worldwide['sum_LUC_to_c3ann'].isel(time=time_idx).any()), average_density_natveg_bio[time_idx], 0)
    print(carbon_density_natveg_bio)

In [ ]:

#aku ga yakin di line 333 gmn cara menerapkan luh2_trans_worldwide['sum_LUC_to_c3ann'] ke semua kelas, apakah kita pre-processing diawal scr batch, atau dimasing2 kategori crops

#tolong assess line 345-387, karena struktur carbon_density nya berbeda (agregat semua primf, secdf, primn, secdn). Pertanyannya sama spt yang lalu, apakah
#total all_to_crops_bio itu equal dengan sum all_to_c3ann_bio + semua kelas?
#apakah temp_LUC_crops_increment_biomass equal dengan all_crops_to_bio?


for time_idx in range(max_year):
    for j, colj in enumerate(natveg_to_crops):
        #c3ann
        temp_carbon_emit_trans_LUC_crops_bio[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c3ann'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_bio[time_idx]))
        if j == 0:
            all_to_c3ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][j]
            all_to_crops_bio[time_idx] += all_to_c3ann_bio[time_idx]

        #c4ann
        temp_carbon_emit_trans_LUC_crops_bio[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c4ann'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_bio[time_idx]))
        if j == 1:
            all_to_c4ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][j]
            all_to_crops_bio[time_idx] += all_to_c4ann_bio[time_idx]

        #c3per
        temp_carbon_emit_trans_LUC_crops_bio[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c3per'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_bio[time_idx]))
        if j == 2:
            all_to_c3per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][j]
            all_to_crops_bio[time_idx] += all_to_c3per_bio[time_idx]

        #c4per
        temp_carbon_emit_trans_LUC_crops_bio[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c4per'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_bio[time_idx]))
        if j == 3:
            all_to_c4per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][j]
            all_to_crops_bio[time_idx] += all_to_c4per_bio[time_idx]

        #c3nfx
        temp_carbon_emit_trans_LUC_crops_bio[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c3nfx'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_bio[time_idx]))
        if j == 4:
            all_to_c3nfx_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][j]
            all_to_crops_bio[time_idx] += all_to_c3nfx_bio[time_idx]

        temp_LUC_crops_increment_biomass[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][j]



In [ ]:
##1b PASTURE TO CROP BIOMASS

##2a1 FOREST TO PASTURE BIOMASS

selisih_carbon_density = np.zeros((max_year, 720, 1440), dtype="float32")

#isi gap data natveg - natural vegetation (forest)
for time_idx in range(max_year):
    for x in range(720):
        for y in range (1440):
            if ccode_convert[x][y] != "Ocean":
                if not np.isnan(luh2_trans_worldwide['sum_LUC_pastures'].isel(time=time_idx, lat=x, lon=y).values) and (luh2_trans_worldwide['sum_LUC_pastures'].isel(time=time_idx, lat=x, lon=y).values != 0):
                    if carbon_density_natveg_bio[time_idx][x][y] == 0:
                        carbon_density_natveg_bio[time_idx][x][y] = fill_nan_with_nearest(natveg_all_carbon_density['natveg_all_interp'].isel(time=time_idx).sel(NBP=carstock_biomass))

#if condition kedua cukup spesifik, bukan NaN (laut), dan bukan 0 (didarat tapi memang tidak ada land use change ke pasture)
                        #dalam kata lain luh2_trans_worldwide['sum_LUC_pastures'] ada valuenya selain NaN dan 0

#isi gap data carbon density pasture dengan data range
#konsep: jika ada perubahan forest ke pasture (let say dari 1970 ke 1971), maka harus ada nilai carbon density pasture di tahun 1971 (time_idx+1)
    #jika nilainya tidak ada maka fillna dengan nilai nearest dengan toleransi 32 pixel
        #jika masih tidak ada juga, maka kita kalkulasi rata2 carbon density pasture di negara tersebut

for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for x in range(720):
            for y in range (1440):
                if ccode_convert[x][y] != "Ocean":
                    if not np.isnan(luh2_trans_worldwide['sum_LUC_pastures'].isel(time=time_idx, lat=x, lon=y).values) and (luh2_trans_worldwide['sum_LUC_pastures'].isel(time=time_idx, lat=x, lon=y).values != 0):
                        if carbon_density_pastr_bio[time_idx + 1][x][y] == 0:
                            carbon_density_pastr_bio[time_idx + 1][x][y].fillna(carbon_density_range_bio[time_idx + 1][x][y])

                            #kalau dari range juga engga ada valuenya
                            if carbon_density_pastr_bio[time_idx + 1][x][y] == 0:
                                carbon_density_pastr_bio[time_idx + 1][x][y] = fill_nan_with_nearest(pastr_carbon_density['pastr_interp'].isel(time=time_idx).sel(NBP=carstock_biomass))

for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        area_changed_forest_to_pasture = np.multiply.reduce(luh2_static['carea'].to_numpy(),
                                            np.nan_to_num(luh2_trans_worldwide['sum_LUC_pastures'].isel(time=time_idx).to_numpy()))
        selisih_carbon_density[time_idx] = (carbon_density_natveg_bio[time_idx].to_numpy())-(carbon_density_pastr_bio[time_idx+1].to_numpy())
        temp_LUC_pasture_increment_biomass[time_idx] = selisih_carbon_density * area_changed_forest_to_pasture

##2a2 FOREST TO RANGE BIOMASS
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        temp_LUC_range_increment_biomass[time_idx] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_transfrom'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_bio[time_idx]))
##2b CROP TO PASTURE BIOMASS


In [ ]:
##3 Land Abandonment to SECDF BIOMASS

###SOIL
#Assign the carbon density of soil value to each year
for time_idx in range(max_year):
    for soil_class in carstock_soil:
        carbon_density_natveg_soil[time_idx] += natveg_all_carbon_density['natveg_all_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c3ann_soil[time_idx] += c3ann_carbon_density['CROPc3ann_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c4ann_soil[time_idx] += c4ann_carbon_density['CROPc4ann_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c3per_soil[time_idx] += c3per_carbon_density['CROPc3per_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c4per_soil[time_idx] += c4per_carbon_density['CROPc4per_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c3nfx_soil[time_idx] += c3nfx_carbon_density['CROPc3nfx_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_pastr_soil[time_idx] += pastr_carbon_density['pastr_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_range_soil[time_idx] += range_carbon_density['range_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()

##1a FOREST TO CROP SOIL
natveg_to_crops = ['sum_LUC_to_c3ann','sum_LUC_to_c4ann','sum_LUC_to_c3per','sum_LUC_to_c4per','sum_LUC_to_c3nfx']

for time_idx in range(max_year):
    for j, colj in enumerate(natveg_to_crops):
        #c3ann
        temp_carbon_emit_trans_LUC_crops_soil[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c3ann'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_soil[time_idx]))
        if j == 0:
            all_to_c3ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][j]
            all_to_crops_soil[time_idx] += all_to_c3ann_soil[time_idx]

        #c4ann
        temp_carbon_emit_trans_LUC_crops_soil[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c4ann'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_soil[time_idx]))
        if j == 1:
            all_to_c4ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][j]
            all_to_crops_soil[time_idx] += all_to_c4ann_soil[time_idx]

        #c3per
        temp_carbon_emit_trans_LUC_crops_soil[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c3per'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_soil[time_idx]))
        if j == 2:
            all_to_c3per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][j]
            all_to_crops_soil[time_idx] += all_to_c3per_soil[time_idx]

        #c4per
        temp_carbon_emit_trans_LUC_crops_soil[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c4per'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_soil[time_idx]))
        if j == 3:
            all_to_c4per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][j]
            all_to_crops_soil[time_idx] += all_to_c4per_soil[time_idx]

        #c3nfx
        temp_carbon_emit_trans_LUC_crops_soil[time_idx][j] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_to_c3nfx'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_soil[time_idx]))
        if j == 4:
            all_to_c3nfx_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][j]
            all_to_crops_soil[time_idx] += all_to_c3nfx_soil[time_idx]

        temp_LUC_crops_increment_biomass[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][j]


##2a1 FOREST TO PASTURE SOIL
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        #pasture
        temp_LUC_pasture_increment_soil[time_idx] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_pastures'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_soil[time_idx]))
##2a2 FOREST TO RANGE SOIL
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        #pasture
        temp_LUC_range_increment_soil[time_idx] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide['sum_LUC_transfrom'].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_natveg_soil[time_idx]))
